In [5]:
from collections import defaultdict

import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Загрузка данных

В качестве данных возьмем стаднадртный датасет ирисов. Подробное описание можно прочитать здесь:
https://ru.xcv.wiki/wiki/Iris_flower_data_set

In [9]:
data = load_iris()

X, y = pd.DataFrame(data.data), pd.DataFrame(data.target)

In [14]:
X.head(10)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


# Разбор работы алгоритма

Теорию о том как работает наивный баейсовский классификатор можно прочитать здесь: https://nuancesprog.ru/p/10732/

# Реализация на python

Решим задачу с классификацией ирисов (по мотивам статьи https://shwanoff.ru/наивный-байесовский-классификатор-н/)

In [2]:
class NaiveBayesClassifier(object):
    def init(self):  
        self.__class_freq = defaultdict(lambda:0)
        self.__feat_freq = defaultdict(lambda:0)


    def fit(self, X, y):
        # calculate classes and features frequencies
        for feature, label in zip(X, y):
            self.__class_freq[label] += 1
            for value in feature:
                self.__feat_freq[(value, label)] += 1

        # normalizate values
        num_samples = len(X)
        for k in self.__class_freq:
            self.__class_freq[k] /= num_samples

        for value, label in self.__feat_freq:
            self.__feat_freq[(value, label)] /= self.__class_freq[label]

        return self
    
    
    def predict(self, X):
    # return argmin of classes 
    return min(self.__class_freq.keys(), 
               key=lambda c : self.__calculate_class_freq(X, c)) 
 
    def __calculate_class_freq(self, X, clss):
        # calculate frequence for current class
        freq = - np.log(self.__class_freq[clss])

        for feat in X: 
            freq += - np.log(self.__feat_freq.get((feat, clss), 10 ** (-7)))
        return freq

Разобьем нашу выборку на обучающую и тестовую, обучим классификатор и посмотрим результаты

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=21414)

model = NaiveBayesClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(accuracy_score(predictions, y_test))

0.9666666666666667


Получили отличную точность. Но здесь больше заслуга хорошего датасета, а не самого алгоритма. Для сравнения можно рассмотреть задачу из статьи ниже:

Пример про наивный байесовский классификатор для классификации диабета: https://www.machinelearningmastery.ru/naive-bayes-classifier-scratch-python/ (реализован на python-2)